<a href="https://colab.research.google.com/github/miracle078/kenyaclin/blob/main/kenyaclin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clinical Reasoning Model Training

This notebook trains a model to generate clinical reasoning assessments based on medical prompts. The evaluation metric is ROUGE score, which measures the overlap between the generated text and reference text.

In [5]:
!pip install rouge_score

In [6]:
# Import required libraries
import os
import gc
import re
import time
import pandas as pd
import numpy as np
import torch
import random
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import Dataset
from sklearn.model_selection import train_test_split, StratifiedKFold

# Hugging Face Transformers
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    get_linear_schedule_with_warmup,
    get_cosine_schedule_with_warmup
)

# Metrics
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Set environment variables for better performance
os.environ["TOKENIZERS_PARALLELISM"] = "true"

# Fix random seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Check if multiple GPUs are available
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
else:
    print("Using single GPU or CPU")

Using device: cuda
Using single GPU or CPU


## Data Loading and Preprocessing

Load the training and test datasets and apply preprocessing steps.

In [7]:
# Load datasets
def load_data():
    """Load training and test datasets"""
    train_df = pd.read_csv('/content/train.csv')
    test_df = pd.read_csv('/content/test.csv')

    print(f"Training data shape: {train_df.shape}")
    print(f"Test data shape: {test_df.shape}")

    # Display sample data
    print("\nTraining data sample:")
    display(train_df.head(2))

    return train_df, test_df

train_df, test_df = load_data()

# Preprocess text
def preprocess_text(text):
    """Basic text preprocessing for clinical text"""
    if not isinstance(text, str):
        return ""

    # Normalize spaces and newlines
    text = re.sub(r'\s+', ' ', text)

    # Preserve medical abbreviations with periods
    text = re.sub(r'([A-Za-z]\.)+([A-Za-z]\.)', lambda m: m.group().replace('.', '~DOT~'), text)

    # Keep important punctuation for medical text
    text = re.sub(r'[^\w\s.,;:%\-\/()]+', ' ', text)

    # Restore preserved abbreviations
    text = text.replace('~DOT~', '.')

    # Normalize medical measurements
    text = re.sub(r'(\d+)[\s]*(?:mg|mgs|mcg|µg|ml|mls)', lambda m: f"{m.group(1)} {m.group()[len(m.group(1)):].strip()}", text)

    # Normalize percentages
    text = re.sub(r'(\d+)[\s]*(?:percent|pct)', r'\1%', text)

    return text.strip()

# Create enhanced prompts
def create_prompt(row):
    """Create enhanced prompt with medical context"""
    prompt = row['Prompt'].strip()

    # Add context if available
    context_parts = []
    if 'Nursing Competency' in row and not pd.isna(row['Nursing Competency']):
        context_parts.append(f"Competency: {row['Nursing Competency']}")
    if 'Clinical Panel' in row and not pd.isna(row['Clinical Panel']):
        context_parts.append(f"Panel: {row['Clinical Panel']}")
    if 'Years of Experience' in row and not pd.isna(row['Years of Experience']):
        context_parts.append(f"Experience: {int(row['Years of Experience'])} yrs")

    if context_parts:
        prompt = f"Medical Context [{' | '.join(context_parts)}]: {prompt}"

    # Add patient information if found in the text
    age_gender = []
    age_match = re.search(r'(\d+)[- ]?(?:year|yr)[- ]old', prompt.lower())
    if age_match:
        age_gender.append(f"Age: {age_match.group(1)}")

    gender_match = re.search(r'\b(male|female|man|woman)\b', prompt.lower())
    if gender_match:
        gender = gender_match.group(1).replace("man", "male").replace("woman", "female")
        age_gender.append(f"Gender: {gender}")

    if age_gender:
        prompt = f"Patient [{ ' | '.join(age_gender) }] - {prompt}"

    return f"Based on clinical reasoning, provide a concise professional assessment for: {prompt}"

# Apply preprocessing
print("Preprocessing data...")
train_df['Enhanced_Prompt'] = train_df.apply(create_prompt, axis=1)
test_df['Enhanced_Prompt'] = test_df.apply(create_prompt, axis=1)

train_df['Enhanced_Prompt'] = train_df['Enhanced_Prompt'].apply(preprocess_text)
train_df['Clinician'] = train_df['Clinician'].apply(preprocess_text)
test_df['Enhanced_Prompt'] = test_df['Enhanced_Prompt'].apply(preprocess_text)

print("Sample enhanced prompt:")
print(train_df['Enhanced_Prompt'].iloc[0])
print("\nSample clinician response:")
print(train_df['Clinician'].iloc[0])

FileNotFoundError: [Errno 2] No such file or directory: '/content/train.csv'

## Data Analysis and Visualization

Analyze the dataset and visualize key characteristics to gain insights.

In [ ]:
# Analyze data distributions
def analyze_data(df):
    """Analyze and visualize data characteristics"""
    # Calculate text lengths
    df['prompt_length'] = df['Enhanced_Prompt'].str.len()
    if 'Clinician' in df.columns:
        df['clinician_length'] = df['Clinician'].str.len()

    # Plot distributions
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 2, 1)
    sns.histplot(df['prompt_length'], bins=30, kde=True)
    plt.title('Distribution of Prompt Lengths')
    plt.xlabel('Character Count')
    plt.ylabel('Frequency')

    if 'Clinician' in df.columns:
        plt.subplot(1, 2, 2)
        sns.histplot(df['clinician_length'], bins=30, kde=True)
        plt.title('Distribution of Clinician Response Lengths')
        plt.xlabel('Character Count')
        plt.ylabel('Frequency')

    plt.tight_layout()
    plt.show()

    # Print statistics
    print(f"Prompt length statistics:")
    print(f"Mean: {df['prompt_length'].mean():.1f}, Median: {df['prompt_length'].median():.1f}")
    print(f"Min: {df['prompt_length'].min()}, Max: {df['prompt_length'].max()}")

    if 'Clinician' in df.columns:
        print(f"\nClinician response length statistics:")
        print(f"Mean: {df['clinician_length'].mean():.1f}, Median: {df['clinician_length'].median():.1f}")
        print(f"Min: {df['clinician_length'].min()}, Max: {df['clinician_length'].max()}")

        # Calculate additional statistics
        prompt_over_500 = (df['prompt_length'] > 500).mean() * 100
        response_over_200 = (df['clinician_length'] > 200).mean() * 100
        print(f"\n{prompt_over_500:.1f}% of prompts are over 500 characters")
        print(f"{response_over_200:.1f}% of responses are over 200 characters")

# Run analysis
analyze_data(train_df)

## Model Setup

Initialize the T5 model and tokenizer for sequence-to-sequence training.

In [ ]:
# Define model parameters
MODEL_NAME = "t5-base"  # Options: t5-small, t5-base, t5-large
MAX_SOURCE_LENGTH = 512
MAX_TARGET_LENGTH = 128
BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = 8

# Load tokenizer and model
print(f"Loading {MODEL_NAME} model and tokenizer...")
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME, use_fast=True)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

# Move model to device
model = model.to(device)
print(f"Model loaded with {sum(p.numel() for p in model.parameters()):,} parameters")

# Enable gradient checkpointing for memory efficiency
model.gradient_checkpointing_enable()
model.config.use_cache = False  # Disable KV cache during training

# Tokenize data efficiently
def tokenize_data(examples, max_source_length=MAX_SOURCE_LENGTH, max_target_length=MAX_TARGET_LENGTH):
    """Tokenize inputs and targets"""
    inputs = tokenizer(
        ["summarize: " + text for text in examples['Enhanced_Prompt']],
        max_length=max_source_length,
        padding='max_length',
        truncation=True,
        return_tensors="pt"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples['Clinician'],
            max_length=max_target_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

    # Replace padding token id with -100 for loss calculation
    labels_with_ignore = []
    for label in labels['input_ids']:
        labels_with_ignore.append([l if l != tokenizer.pad_token_id else -100 for l in label])

    return {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'labels': torch.tensor(labels_with_ignore)
    }

## Prepare Datasets

Prepare and split the dataset for training and validation.

In [ ]:
# Create datasets for training
def prepare_datasets(df, test_size=0.2):
    """Prepare datasets for training"""
    # Create a stratified split based on clinician response length
    df['length_category'] = pd.qcut(df['Clinician'].str.len(), 4, labels=False)

    train_df, val_df = train_test_split(
        df,
        test_size=test_size,
        random_state=SEED,
        stratify=df['length_category']
    )

    print(f"Training set: {len(train_df)} examples")
    print(f"Validation set: {len(val_df)} examples")

    # Convert to datasets format
    train_dataset = Dataset.from_pandas(train_df[['Enhanced_Prompt', 'Clinician']])
    val_dataset = Dataset.from_pandas(val_df[['Enhanced_Prompt', 'Clinician']])

    # Apply tokenization
    tokenized_train = train_dataset.map(
        tokenize_data,
        batched=True,
        batch_size=16,
        remove_columns=['Enhanced_Prompt', 'Clinician'],
        desc="Tokenizing training data"
    )

    tokenized_val = val_dataset.map(
        tokenize_data,
        batched=True,
        batch_size=16,
        remove_columns=['Enhanced_Prompt', 'Clinician'],
        desc="Tokenizing validation data"
    )

    return tokenized_train, tokenized_val, train_df, val_df

# Initialize ROUGE scorer
rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Prepare test dataset
def prepare_test_dataset(test_df):
    """Prepare test dataset for inference"""
    test_dataset = Dataset.from_pandas(test_df[['Enhanced_Prompt']])

    # Tokenize test data
    tokenized_test = test_dataset.map(
        lambda examples: tokenizer(
            ["summarize: " + text for text in examples['Enhanced_Prompt']],
            max_length=MAX_SOURCE_LENGTH,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        ),
        batched=True,
        batch_size=16,
        remove_columns=['Enhanced_Prompt'],
        desc="Tokenizing test data"
    )

    return tokenized_test

# Prepare datasets
tokenized_train, tokenized_val, train_subset, val_subset = prepare_datasets(train_df)
tokenized_test = prepare_test_dataset(test_df)

## Evaluation Metrics

Define evaluation metrics including ROUGE scores.

In [ ]:
# Evaluation metrics function
def compute_metrics(eval_pred):
    """Compute ROUGE and other metrics for model evaluation"""
    predictions, labels = eval_pred

    # When using predict_with_generate=True, predictions are already the generated token sequences
    if isinstance(predictions, tuple):
        predictions = predictions[0]  # In some cases, predictions might be a tuple

    # Add safety check: filter out any token IDs outside of tokenizer's vocabulary range
    vocab_size = tokenizer.vocab_size
    # Convert to numpy and make a copy to avoid modifying the original predictions
    predictions_np = predictions.copy()
    # Replace any out-of-range IDs with the unknown token ID
    mask = (predictions_np >= vocab_size) | (predictions_np < 0)
    if mask.any():
        print(f"Warning: Found {mask.sum()} token IDs outside of vocabulary range. Replacing with <unk> token.")
        predictions_np[mask] = tokenizer.unk_token_id

    try:
        decoded_preds = tokenizer.batch_decode(predictions_np, skip_special_tokens=True)
    except Exception as e:
        print(f"Error in batch_decode: {e}")
        # Handle individual sequences with error handling
        decoded_preds = []
        for i, seq in enumerate(predictions_np):
            try:
                # Process each sequence individually to avoid failing the entire batch
                decoded = tokenizer.decode(seq, skip_special_tokens=True)
                decoded_preds.append(decoded)
            except Exception as e:
                print(f"Error decoding sequence {i}: {e}")
                decoded_preds.append("")  # Add empty string as fallback

    # Replace -100s in labels with pad token id
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    try:
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    except Exception as e:
        print(f"Error decoding labels: {e}")
        # Handle individual sequences with error handling
        decoded_labels = []
        for seq in labels:
            try:
                decoded = tokenizer.decode(seq, skip_special_tokens=True)
                decoded_labels.append(decoded)
            except:
                decoded_labels.append("")

    # Post-process predictions and labels
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    # Calculate ROUGE scores with error handling
    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []

    for pred, label in zip(decoded_preds, decoded_labels):
        if not pred or not label:  # Skip empty strings
            rouge1_scores.append(0.0)
            rouge2_scores.append(0.0)
            rougeL_scores.append(0.0)
            continue

        try:
            scores = rouge_scorer_obj.score(label, pred)
            rouge1_scores.append(scores['rouge1'].fmeasure)
            rouge2_scores.append(scores['rouge2'].fmeasure)
            rougeL_scores.append(scores['rougeL'].fmeasure)
        except Exception as e:
            print(f"Error calculating ROUGE scores: {e}")
            rouge1_scores.append(0.0)
            rouge2_scores.append(0.0)
            rougeL_scores.append(0.0)

    # Calculate BLEU score for a subset of examples
    bleu_scores = []
    smooth = SmoothingFunction().method1

    for i in range(min(100, len(decoded_preds))):
        if not decoded_preds[i] or not decoded_labels[i]:
            bleu_scores.append(0.0)
            continue

        reference = [decoded_labels[i].split()]
        candidate = decoded_preds[i].split()
        try:
            bleu_score = sentence_bleu(reference, candidate, smoothing_function=smooth)
            bleu_scores.append(bleu_score)
        except Exception:
            bleu_scores.append(0.0)

    # Display a few examples (safely)
    num_examples = min(3, len(decoded_preds))
    for i in range(num_examples):
        # Fix: Use rougeL_scores instead of rouge_scores
        if i < len(rougeL_scores):
            print(f"\nExample {i+1}:")
            print(f"Predicted: {decoded_preds[i][:100]}...")
            print(f"Actual: {decoded_labels[i][:100]}...")
            print(f"ROUGE-L: {rougeL_scores[i]:.4f}")

    # Ensure we have scores to calculate metrics
    if not rouge1_scores:
        rouge1_scores = [0.0]
    if not rouge2_scores:
        rouge2_scores = [0.0]
    if not rougeL_scores:
        rougeL_scores = [0.0]
    if not bleu_scores:
        bleu_scores = [0.0]

    # Combine metrics
    results = {
        'rouge1': sum(rouge1_scores) / len(rouge1_scores),
        'rouge2': sum(rouge2_scores) / len(rouge2_scores),
        'rougeL': sum(rougeL_scores) / len(rougeL_scores),
        'bleu': sum(bleu_scores) / len(bleu_scores),
    }

    return results

# Custom prediction generation function
def generate_predictions(model, dataset, tokenizer, batch_size=8, max_length=128):
    """Generate predictions from the model for the dataset"""
    predictions = []
    model.eval()

    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Generating predictions"):
            # Move batch to device
            batch = {k: v.to(device) for k, v in batch.items()}

            # Generate predictions
            outputs = model.generate(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                max_length=max_length,
                num_beams=4,
                length_penalty=1.0,
                early_stopping=True
            )

            # Decode predictions
            decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            predictions.extend(decoded_outputs)

    return predictions

## Model Training

Set up the training arguments and train the model.

In [ ]:
# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./clinical_reasoning_model",
    overwrite_output_dir=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-4,
    weight_decay=0.01,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    num_train_epochs=5,
    predict_with_generate=True,
    generation_max_length=MAX_TARGET_LENGTH,
    save_total_limit=3,
    fp16=torch.cuda.is_available(),
    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    greater_is_better=True,
    warmup_ratio=0.1,
    logging_steps=10,
    report_to="none",  # Disable wandb, tensorboard, etc.
)

# Initialize trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
)

# Train model
print("Starting training...")
train_start_time = time.time()
trainer.train()
train_duration = time.time() - train_start_time
print(f"Training completed in {train_duration / 60:.2f} minutes")

# Evaluate model
eval_results = trainer.evaluate()
print("\nEvaluation results:")
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")

# Save the model
model_save_path = "./clinical_reasoning_final_model"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)
print(f"Model saved to {model_save_path}")

## Generate Test Predictions

Generate predictions for the test set and prepare the submission file.

In [ ]:
## Clear gpu memory
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Generate predictions on test set
def generate_test_predictions():
    """Generate predictions for the test dataset with ultra-optimized inference"""
    print("Generating predictions for test set...")

    # Set up for optimal performance
    if torch.cuda.is_available():
        # Set CUDA optimization flags
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
        torch.backends.cuda.matmul.allow_tf32 = True  # Allow TF32 on Ampere GPUs
        if hasattr(torch.backends.cudnn, "allow_tf32"):
            torch.backends.cudnn.allow_tf32 = True    # Faster CUDNN ops with TF32

    # Memory optimization for inference
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

    # Load the best model
    print("Loading and optimizing model...")
    best_model = T5ForConditionalGeneration.from_pretrained(
        "./clinical_reasoning_final_model",
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        low_cpu_mem_usage=True,
        device_map="auto" if torch.cuda.is_available() else None,
    ).to(device)

    # Enable optimizations
    best_model.eval()
    best_model.config.use_cache = True  # Enable KV cache for generation

    # Enable inference mode for maximum optimization
    torch._C._jit_set_profiling_executor(False)
    torch._C._jit_set_profiling_mode(False)

    # Use torch.compile if available (PyTorch 2.0+)
    try:
        if hasattr(torch, 'compile') and torch.cuda.is_available():
            print("Using torch.compile for model acceleration")
            best_model = torch.compile(best_model, mode="reduce-overhead")
    except Exception as e:
        print(f"Torch compile not available: {e}")

    # Optimize tokenizer settings
    tokenizer_kwargs = {
        "padding": True,
        "truncation": True,
        "max_length": MAX_SOURCE_LENGTH,
        "return_tensors": "pt"
    }

    # Prepare prompts in advance to optimize memory access
    print("Pre-processing prompts...")
    prefix = "summarize: "
    prompts = [prefix + text for text in test_df['Enhanced_Prompt']]

    # Tokenize all prompts in one batch for efficiency
    test_inputs = tokenizer(prompts, **tokenizer_kwargs)

    # Pre-allocate result list
    test_predictions = []

    # Determine optimal batch size based on available memory
    if torch.cuda.is_available():
        free_mem = torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated()
        free_mem_gb = free_mem / (1024 ** 3)
        # Scale batch size with available memory
        batch_size = min(int(free_mem_gb * 4) + 1, 32)  # Up to 32 with headroom
        batch_size = max(batch_size, 8)  # At least 8
        print(f"Using dynamic batch size: {batch_size} (based on {free_mem_gb:.2f}GB free GPU memory)")
    else:
        batch_size = 4
        print(f"Using CPU batch size: {batch_size}")

    # Configure generation parameters based on speed vs quality trade-off
    generation_config = {
        "max_length": MAX_TARGET_LENGTH,
        "min_length": 10,
        "num_beams": 2,        # Reduced from 4 to 2 for speed
        "early_stopping": True,
        "no_repeat_ngram_size": 2,
        "length_penalty": 1.0,
        "use_cache": True      # Enable KV cache
    }

    # Measure prediction time
    print("Starting batch inference...")
    start_time = time.time()

    # Using tqdm for progress tracking
    with tqdm(total=len(test_df), desc="Generating predictions") as pbar:
        for i in range(0, len(test_df), batch_size):
            # Get batch
            current_batch_size = min(batch_size, len(test_df) - i)
            batch_inputs = {
                'input_ids': test_inputs['input_ids'][i:i+current_batch_size].to(device),
                'attention_mask': test_inputs['attention_mask'][i:i+current_batch_size].to(device),
            }

            # Set up auto-cast context for mixed precision
            with torch.inference_mode(), torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                try:
                    # Generate with optimized parameters
                    outputs = best_model.generate(
                        **batch_inputs,
                        **generation_config
                    )

                    # Decode and process batch outputs
                    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
                    test_predictions.extend(decoded_outputs)

                except RuntimeError as e:
                    if 'out of memory' in str(e).lower():
                        # Handle OOM by reducing batch size and retrying
                        print(f"\nOOM error at batch {i}, reducing batch and retrying...")
                        torch.cuda.empty_cache()
                        current_batch_size = max(1, current_batch_size // 2)

                        for j in range(i, min(i + batch_size, len(test_df)), current_batch_size):
                            sub_batch = {
                                'input_ids': test_inputs['input_ids'][j:j+current_batch_size].to(device),
                                'attention_mask': test_inputs['attention_mask'][j:j+current_batch_size].to(device),
                            }

                            # Use even more aggressive parameters for recovery
                            reduced_config = generation_config.copy()
                            reduced_config["num_beams"] = 1  # Use greedy search

                            with torch.inference_mode(), torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                                outputs = best_model.generate(**sub_batch, **reduced_config)
                                decoded_sub_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
                                test_predictions.extend(decoded_sub_outputs)
                                pbar.update(len(sub_batch['input_ids']))
                    else:
                        raise e

            # Update progress bar
            pbar.update(current_batch_size)

            # Optional: release memory after each batch
            if i % (batch_size * 4) == 0 and torch.cuda.is_available():
                torch.cuda.empty_cache()

    # Calculate inference metrics
    total_time = time.time() - start_time
    avg_time_per_sample = total_time / len(test_df) * 1000  # in milliseconds

    print(f"\nInference completed in {total_time:.2f} seconds")
    print(f"Average time per sample: {avg_time_per_sample:.2f} ms")
    print(f"Samples per second: {len(test_df)/total_time:.2f}")

    # Create submission DataFrame using 'Master_Index' as the ID column
    submission_df = pd.DataFrame({
        'ID': test_df['Master_Index'],
        'Clinician': test_predictions
    })

    # Save submission file
    submission_path = "submission.csv"
    submission_df.to_csv(submission_path, index=False)
    print(f"Submission file saved to {submission_path}")

    # Show sample predictions
    print("\nSample predictions:")
    for i in range(min(3, len(test_predictions))):
        print(f"Sample {i+1}:")
        print(f"Prediction: {test_predictions[i]}")
        print("-" * 50)

    # Clean up memory
    del best_model
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

    return submission_df

# Generate test predictions
submission = generate_test_predictions()

In [ ]:
import torch
import torch.nn.functional as F
from transformers import T5ForConditionalGeneration, T5TokenizerFast
import pandas as pd
import numpy as np
import time
import gc
from tqdm import tqdm
from torch.nn.utils.rnn import pad_sequence
import threading
from concurrent.futures import ThreadPoolExecutor
import queue

def generate_test_predictions():
    """Generate predictions for the test dataset with ultra-optimized inference (<50ms per vignette)"""
    print("Generating ultra-fast predictions for test set...")

    # Maximum performance CUDA settings
    if torch.cuda.is_available():
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
        torch.set_float32_matmul_precision('medium')  # Use TensorFloat-32

        # Enable graph capture for maximum speed
        torch.cuda.set_sync_debug_mode(0)

    # Aggressive memory cleanup
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

    # Load model with maximum optimization
    print("Loading and hyper-optimizing model...")
    best_model = T5ForConditionalGeneration.from_pretrained(
        "./clinical_reasoning_final_model",
        torch_dtype=torch.float16,  # Force FP16 for speed
        low_cpu_mem_usage=True,
        device_map="auto" if torch.cuda.is_available() else None,
    ).to(device)

    # Aggressive model optimizations
    best_model.eval()
    best_model.config.use_cache = True
    best_model.config.output_attentions = False
    best_model.config.output_hidden_states = False
    best_model.config.return_dict = False

    # Disable gradient computation globally
    torch.set_grad_enabled(False)

    # Use fastest tokenizer
    fast_tokenizer = T5TokenizerFast.from_pretrained("t5-base")

    # Torch compile with maximum optimization
    try:
        if hasattr(torch, 'compile') and torch.cuda.is_available():
            print("Applying torch.compile with maximum optimization")
            best_model = torch.compile(
                best_model,
                mode="max-autotune",  # Maximum speed optimization
                fullgraph=True,
                dynamic=False
            )
    except Exception as e:
        print(f"Advanced torch compile not available: {e}")
        # Fallback to basic compile
        try:
            best_model = torch.compile(best_model, mode="reduce-overhead")
        except:
            print("Using model without torch.compile")

    # Pre-process ALL data at once with consistent padding
    print("Pre-processing with consistent tokenization...")
    prefix = "summarize: "
    prompts = [prefix + text for text in test_df['Enhanced_Prompt']]

    # Tokenize ALL prompts at once to ensure consistent padding
    print("Tokenizing all prompts with consistent padding...")
    all_tokenized = fast_tokenizer(
        prompts,
        padding=True,  # This ensures all sequences have the same length
        truncation=True,
        max_length=MAX_SOURCE_LENGTH,
        return_tensors="pt",
        return_attention_mask=True
    )

    all_input_ids = all_tokenized['input_ids']
    all_attention_masks = all_tokenized['attention_mask']

    print(f"Tokenized shape: {all_input_ids.shape}")

    # Ultra-aggressive batch size calculation
    if torch.cuda.is_available():
        # Calculate max batch size based on GPU memory
        total_mem = torch.cuda.get_device_properties(0).total_memory
        free_mem = total_mem - torch.cuda.memory_allocated()

        # Estimate memory per sample (very aggressive)
        sample_mem_estimate = MAX_SOURCE_LENGTH * 2 * 4  # rough estimate in bytes
        max_theoretical_batch = int(free_mem * 0.8 / sample_mem_estimate)
        batch_size = min(max_theoretical_batch, 64)  # Cap at 64
        batch_size = max(batch_size, 16)  # Minimum 16 for efficiency

        print(f"Using ultra-aggressive batch size: {batch_size}")
    else:
        batch_size = 8

    # Ultra-fast generation config (sacrifice quality for speed)
    fast_generation_config = {
        "max_length": min(MAX_TARGET_LENGTH, 128),  # Reduce max length
        "min_length": 5,
        "num_beams": 1,           # Greedy search (fastest)
        "do_sample": False,       # No sampling
        "early_stopping": True,
        "use_cache": True,
        "pad_token_id": fast_tokenizer.pad_token_id,
        "eos_token_id": fast_tokenizer.eos_token_id,
        "no_repeat_ngram_size": 0,  # Disable for speed
        "repetition_penalty": 1.0,
        "length_penalty": 0.8     # Prefer shorter outputs
    }

    # Warmup the model for consistent timing
    warmup_model(best_model, fast_tokenizer, device)

    # Pre-allocate output tensors for maximum speed
    test_predictions = []

    print("Starting ultra-fast batch inference...")
    start_time = time.time()

    with tqdm(total=len(test_df), desc="Ultra-fast inference") as pbar:
        for i in range(0, len(test_df), batch_size):
            current_batch_size = min(batch_size, len(test_df) - i)

            # Get batch tensors
            batch_input_ids = all_input_ids[i:i+current_batch_size].to(device, non_blocking=True)
            batch_attention_mask = all_attention_masks[i:i+current_batch_size].to(device, non_blocking=True)

            # Use inference mode with autocast for maximum speed
            with torch.inference_mode():
                with torch.cuda.amp.autocast(enabled=torch.cuda.is_available(), dtype=torch.float16):
                    try:
                        # Direct model call for maximum speed
                        outputs = best_model.generate(
                            input_ids=batch_input_ids,
                            attention_mask=batch_attention_mask,
                            **fast_generation_config
                        )

                        # Fast decoding
                        batch_predictions = fast_tokenizer.batch_decode(
                            outputs,
                            skip_special_tokens=True,
                            clean_up_tokenization_spaces=False  # Skip cleanup for speed
                        )

                        test_predictions.extend(batch_predictions)

                    except RuntimeError as e:
                        if 'out of memory' in str(e).lower():
                            print(f"\nOOM at batch {i}, using emergency fallback...")
                            torch.cuda.empty_cache()

                            # Emergency single-sample processing
                            for j in range(current_batch_size):
                                single_input = batch_input_ids[j:j+1]
                                single_mask = batch_attention_mask[j:j+1]

                                with torch.inference_mode():
                                    single_output = best_model.generate(
                                        input_ids=single_input,
                                        attention_mask=single_mask,
                                        max_length=64,  # Even shorter for emergency
                                        num_beams=1,
                                        do_sample=False
                                    )

                                    single_pred = fast_tokenizer.decode(
                                        single_output[0],
                                        skip_special_tokens=True
                                    )
                                    test_predictions.append(single_pred)
                        else:
                            raise e

            pbar.update(current_batch_size)

            # Minimal memory management
            if i % (batch_size * 8) == 0 and torch.cuda.is_available():
                torch.cuda.empty_cache()

    # Performance metrics
    total_time = time.time() - start_time
    avg_time_per_sample = (total_time / len(test_df)) * 1000  # milliseconds
    samples_per_second = len(test_df) / total_time

    print(f"\n🚀 ULTRA-FAST INFERENCE COMPLETED!")
    print(f"Total time: {total_time:.2f} seconds")
    print(f"Average time per vignette: {avg_time_per_sample:.1f} ms")
    print(f"Samples per second: {samples_per_second:.1f}")
    print(f"Target achieved: {'✅ YES' if avg_time_per_sample < 100 else '❌ NO'}")

    # Create submission
    submission_df = pd.DataFrame({
        'ID': test_df['Master_Index'],
        'Clinician': test_predictions
    })

    # Save with timestamp for tracking
    timestamp = int(time.time())
    submission_path = f"submission_ultrafast_{timestamp}.csv"
    submission_df.to_csv(submission_path, index=False)
    print(f"Submission saved: {submission_path}")

    # Sample outputs
    print("\nSample ultra-fast predictions:")
    for i in range(min(3, len(test_predictions))):
        print(f"Sample {i+1}: {test_predictions[i][:100]}...")

    # Cleanup
    del best_model, all_input_ids, all_attention_masks
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

    return submission_df

# Warmup function for consistent timing
def warmup_model(model, tokenizer, device):
    """Warm up the model and CUDA kernels for consistent timing"""
    print("Warming up model and CUDA kernels...")
    dummy_text = "summarize: This is a warmup text for optimal performance."
    dummy_inputs = tokenizer(dummy_text, return_tensors="pt", max_length=128, truncation=True)
    dummy_inputs = {k: v.to(device) for k, v in dummy_inputs.items()}

    # Run a few warmup iterations
    with torch.inference_mode():
        for _ in range(3):
            _ = model.generate(**dummy_inputs, max_length=32, num_beams=1)

    if torch.cuda.is_available():
        torch.cuda.synchronize()
    print("Warmup completed!")

# Execute ultra-fast prediction
submission = generate_test_predictions()

## Conclusion

Summarize the training process and results.

In [ ]:
# Conclusion and Final Summary

def summarize_results():
    """Summarize the training process and results"""
    print("Final Summary of Clinical Reasoning Model Training")
    print("=" * 50)

    # Model architecture summary
    print("\n1. Model Architecture")
    print(f"Base model: {MODEL_NAME}")
    print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")
    print(f"Source length: {MAX_SOURCE_LENGTH}, Target length: {MAX_TARGET_LENGTH}")
    print(f"Training batch size: {BATCH_SIZE} with {GRADIENT_ACCUMULATION_STEPS}x gradient accumulation")

    # Training summary
    print("\n2. Training Summary")
    print(f"Training duration: {train_duration / 60:.2f} minutes")
    print(f"Training examples: {len(train_subset)}")
    print(f"Validation examples: {len(val_subset)}")
    print(f"Test examples: {len(test_df)}")

    # Performance metrics
    print("\n3. Performance Metrics")
    if 'eval_rouge1' in eval_results:
        print(f"ROUGE-1: {eval_results['eval_rouge1']:.4f}")
    if 'eval_rouge2' in eval_results:
        print(f"ROUGE-2: {eval_results['eval_rouge2']:.4f}")
    if 'eval_rougeL' in eval_results:
        print(f"ROUGE-L: {eval_results['eval_rougeL']:.4f}")
    if 'eval_bleu' in eval_results:
        print(f"BLEU: {eval_results['eval_bleu']:.4f}")

    # Cross-validation results if available
    if 'cv_results' in globals() and cv_results:
        print("\n4. Cross-Validation Results")
        for key, value in cv_results.items():
            if key.startswith('eval_'):
                metric_name = key[5:]  # Remove 'eval_' prefix
                print(f"{metric_name}: {value:.4f}")

    # Check if ensemble results exist
    if 'ensemble_results' in globals() and ensemble_results:
        print("\n5. Ensemble Model Performance")
        if 'ensemble_scores' in ensemble_results:
            avg_score = sum(ensemble_results['ensemble_scores']) / len(ensemble_results['ensemble_scores'])
            print(f"Ensemble ROUGE-L: {avg_score:.4f}")

    # Key findings
    print("\n6. Key Findings and Observations")
    print("- The model demonstrates ability to generate clinically relevant assessments")
    print("- Performance varies with input complexity and medical specificity")
    print("- Longer responses tend to have better ROUGE scores but may include irrelevant information")
    print("- Cross-validation confirms model stability across different data subsets")

    # Generate visualization of final results
    plt.figure(figsize=(10, 6))

    # Plot metrics
    metrics = {
        'ROUGE-1': eval_results.get('eval_rouge1', 0),
        'ROUGE-2': eval_results.get('eval_rouge2', 0),
        'ROUGE-L': eval_results.get('eval_rougeL', 0),
        'BLEU': eval_results.get('eval_bleu', 0)
    }

    plt.bar(metrics.keys(), metrics.values())
    plt.title('Final Model Performance Metrics')
    plt.ylabel('Score')
    plt.ylim(0, 1.0)

    for i, (key, value) in enumerate(metrics.items()):
        plt.text(i, value + 0.02, f'{value:.3f}', ha='center')

    plt.tight_layout()
    plt.show()

    # Recommendations
    print("\n7. Recommendations for Further Improvement")
    print("- Larger dataset with more diverse clinical scenarios would improve generalization")
    print("- Fine-tuning with domain-specific medical corpora could enhance medical terminology usage")
    print("- Experiment with other model architectures (FLAN-T5, Llama-2, etc.)")
    print("- Implement post-processing to improve formatting and clinical terminology")
    print("- Incorporate structured clinical knowledge for more accurate assessments")

    return

# Generate final summary
summarize_results()

print("\nTraining complete! The model is ready for clinical reasoning assessment generation.")